In [2]:
import os

## generate DCN form

In [1]:
from transformers import BertTokenizer
tokenizer = BertTokenizer('../vocab/vocab.txt')

/home/chendian/.conda/envs/bbcm/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
text = "到涨价——降价——涨价的“循环”馊主意中去。"
text = "根据美国法典15usc1595a(b)的规定，任何人（包括个人及公司），引导或协助非法的进口行为（包括进口“禁止的危险物品”）应收单罚款，罚款的金额应为进口物品的价值。"
text = "对于第三方受托支付借款业务，公司应当提供的商务合同，发票及其他凭证等相关资料进行合法性及真实性审查。"
tokenizer.tokenize(text)

In [6]:
from pypinyin import lazy_pinyin
tokens = ['今', '天', '30', '度', 'Ｉ']
lazy_pinyin(tokens)

['jin', 'tian', '30', 'du', 'Ｉ']

In [9]:
from tqdm import tqdm
from pypinyin import lazy_pinyin
# err, cor, 111, pinyin_indexes


vocab = [line.strip() for line in open('../vocab/vocab.txt', 'r')]
pinyin_vocab = [line.strip() for line in open('../vocab/pinyin_vocab.txt', 'r')]

"""
英 国 卫 报 今 天 报 导 ， 为 了 避 免 引 发 英 国 国 教 派 的 不 满 ， 一 项 有 关 英 国 女 王 伊 莉 莎 白 二 世 将 在 下 新 期 访 问 梵 蒂 冈 时 ， 和 天 主 教 教 宗 若 望 保 禄 二 世 共 同 举 行 弥 撒 的 计 画 已 经 取 消 。	英 国 卫 报 今 天 报 导 ， 为 了 避 免 引 发 英 国 国 教 派 的 不 满 ， 一 项 有 关 英 国 女 王 伊 莉 莎 白 二 世 将 在 下 星 期 访 问 梵 蒂 冈 时 ， 和 天 主 教 教 宗 若 望 保 禄 二 世 共 同 举 行 弥 撒 的 计 画 已 经 取 消 。	1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1	359 107 333 10 133 318 10 60 0 333 164 14 194 358 80 359 107 107 131 231 61 21 186 0 357 341 362 103 359 107 226 332 357 167 283 7 79 292 130 368 339 344 247 82 334 81 63 92 292 0 113 318 387 131 131 396 274 332 10 178 79 292 98 322 137 345 193 275 61 127 120 357 134 257 342 0
清 华 大 学 段 海 鑫 教 授 表 示 。	清 华 大 学 段 海 新 教 授 表 示 。	1 1 1 1 1 1 1 1 1 1 1 1	254 120 56 350 72 109 344 131 293 16 292 0
"""

def B2Q(uchar):
    """单个字符 半角转全角"""
    inside_code = ord(uchar)
    if inside_code < 0x0020 or inside_code > 0x7e: # 不是半角字符就返回原来的字符
        return uchar 
    if inside_code == 0x0020: # 除了空格其他的全角半角的公式为: 半角 = 全角 - 0xfee0
        inside_code = 0x3000
    else:
        inside_code += 0xfee0
    return chr(inside_code).upper()


def text_to_dcn_form(text, tokenizer):    
    tokens = tokenizer.tokenize(''.join([B2Q(c) for c in text]))
    return tokens


def pinyin_index(pinyin_str):
    try:
        res = pinyin_vocab.index(pinyin_str)
        return res
    except Exception as e:
        return 0  # [UNK]


# cctc
# src_path = '../../data/cn/cctc/cctc_test.tsv'
# tgt_path = '../../data/cn/cctc/cctc_test.dcn.txt'

# dcn
# src_path = '../../data/cn/Wang271k_augw_ime/dcn_train.augw1.tsv'
# tgt_path = '../../data/cn/Wang271k_augw_ime/dcn_train.augw1.dcn.txt'

# rw
# src_path = '../../data/cn/rw/rw_test.tsv'
# tgt_path = '../../data/cn/rw/rw_test.dcn.txt'

# findoc
src_path = '../../data/cn/findoc/findoc_test.v2.tsv'
# tsv_path = '../../data/cn/findoc/findoc_test.v2.tsv.bak'
tgt_path = '../../data/cn/findoc/findoc_test.v2.dcn.txt'

# with open(tsv_path, 'w') as f2:
with open(tgt_path, 'w') as f:
    for idx, line in tqdm(enumerate(open(src_path, 'r'))):
        err, cor = line.strip().split('\t')
        err = ''.join([B2Q(c) for c in err])
        cor = ''.join([B2Q(c) for c in cor])
        tokens = tokenizer.tokenize(err)
        cor_tokens = tokenizer.tokenize(cor)
        try:
            assert sum([o != c for o, c in zip(tokens, cor_tokens)]) < 5
            assert len(err) == len(cor)
            assert len(tokens) == len(cor_tokens)
        except Exception as e:
            print(e)
            print(err)
            print(cor)
            print(list(zip(tokens, cor_tokens)))
            continue
        len_index = ' '.join([str(1) for _ in range(len(tokens))])
        py_indexes = [str(pinyin_index(_py)) for _py in lazy_pinyin(tokens)]
        py_index = ' '.join(py_indexes)
        if len(tokens) == len(cor_tokens) == len(py_indexes):
            res = f"{' '.join(tokens)}\t{' '.join(cor_tokens)}\t{len_index}\t{py_index}\n"
            f.write(res)
            # f2.write(line)

6569it [00:18, 352.21it/s]


In [12]:
# check validity
# tgt_path = '../../data/cn/Wang271k_augc/dcn_train.augc.dcn.txt'
tgt_path = '../../data/cn/rw/rw_test.dcn.txt'

for line_idx, line in enumerate(open(tgt_path, 'r')):
    err, cor, len_index, py_index = line.strip().split('\t')
    err = err.split(' ')
    cor = cor.split(' ')
    len_index = len_index.split(' ')
    py_index = py_index.split(' ')
    if len(err) == len(cor) == len(len_index) == len(py_index):
        continue
    print(line_idx)
    # print(err, cor, len_index, py_index)
else:
    print("End.")

In [ ]:
# drop invalid lines
drop = []
# drop = [19567,51006,198075]

for path in ['../../data/cn/Wang271k_augc/dcn_train.augc.dcn.txt', 
             '../../data/cn/Wang271k_augc/dcn_train.augc.tsv']:
    lines = []
    with open(path, 'r') as f:
        lines = [line for idx, line in enumerate(f)]
        with open(path+'.bak', 'w') as f_bak:
            for line in lines:
                f_bak.write(line)
        lines = [line for idx, line in enumerate(lines) if idx not in drop]
    with open(path, 'w') as f:
        for line in lines:
            f.write(line)
    print(f"{path} done.")

## generate SIGHAN form

In [3]:
def B2Q(uchar):
    """单个字符 半角转全角"""
    inside_code = ord(uchar)
    if inside_code < 0x0020 or inside_code > 0x7e: # 不是半角字符就返回原来的字符
        return uchar 
    if inside_code == 0x0020: # 除了空格其他的全角半角的公式为: 半角 = 全角 - 0xfee0
        inside_code = 0x3000
    else:
        inside_code += 0xfee0
    return chr(inside_code).upper()


def fn(src_path, tgt_path, source='CCTC'):
    items = [line.split('\t')[0] for line in open(src_path, 'r')]
    print(len(items))
    with open(tgt_path, 'w') as f:
        for idx, item in enumerate(items):
            item = ''.join([B2Q(c) for c in item])
            f.write(f"(pid={source}-{idx})\t{item}\n")
    

# src_path = '../data/cctc/cctc_test.txt'
# tgt_path = '../data/cctc/cctc.txt'
# fn(src_path, tgt_path, 'CCTC')

src_path = '../../data/cn/rw/rw_test.tsv'
tgt_path = '../../data/cn/rw/rw_test.sighan.txt'
fn(src_path, tgt_path, 'rw')

1089
